In [1]:
sc

In [2]:
! ls -alh /data/tmp/

total 45G
drwxr-xr-x 2 ubuntu ubuntu 4.0K Oct 19 13:32 .
drwxr-xr-x 6 root   root   4.0K Oct 19 12:20 ..
-rw-rw-r-- 1 ubuntu ubuntu  14K Oct 19 12:59 log
-rw-rw-r-- 1 ubuntu ubuntu 639M Oct 19 13:26 pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 757M Oct 19 13:13 pr_day_BCSD_historical_r1i1p1_bcc-csm1-1_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 745M Oct 19 13:19 pr_day_BCSD_historical_r1i1p1_BNU-ESM_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 773M Oct 19 13:04 pr_day_BCSD_historical_r1i1p1_CanESM2_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 568M Oct 19 13:14 pr_day_BCSD_historical_r1i1p1_CCSM4_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 573M Oct 19 13:06 pr_day_BCSD_historical_r1i1p1_CESM1-BGC_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 622M Oct 19 13:21 pr_day_BCSD_historical_r1i1p1_CNRM-CM5_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 664M Oct 19 13:24 pr_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 666M Oct 19 13:08 pr_day_BCSD_historical_r1i1p1_GFDL-CM3_1997.nc
-rw-rw-r-- 1 u

In [3]:
files = !! ls /data/tmp
files = [f for f in files if f.endswith('nc')]

In [36]:
from netCDF4 import Dataset, num2date

In [5]:
ds = Dataset("/data/tmp/" + files[0], 'r', format='NETCDF4')

In [27]:
import numpy as np

np.shape(ds['pr'])

(365, 720, 1440)

### Example Avro
http://avro.apache.org/docs/1.7.7/gettingstartedpython.html

In [ ]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import json

In [ ]:
python_schema = {
    "namespace": "example.avro",
    "type": "record",
    "name": "User",
    "fields": [
         {"name": "name", "type": "string"},
         {"name": "favorite_number",  "type": ["int", "null"]},
         {"name": "favorite_color", "type": ["string", "null"]}
     ]
}

In [ ]:
schema = avro.schema.parse(json.dumps(python_schema))

In [ ]:
writer = DataFileWriter(open("users.avro", "w"), DatumWriter(), schema)
writer.append({"name": "Alyssa", "favorite_number": 256})
writer.append({"name": "Ben", "favorite_number": 7, "favorite_color": "red"})
writer.close()

In [ ]:
reader = DataFileReader(open("users.avro", "r"), DatumReader())
for user in reader:
    print user
reader.close()

### NetCDF test

In [187]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import json

In [188]:
import time

def netcdf_items(var):
    ds = var._grp
    
    def to_unix_timestamp(i):
        return int(time.mktime(num2date(ds.variables['time'][i], 
                                 ds.variables['time'].units ).timetuple()))

    times, lats, lons = var.shape
    for t in xrange(times):
        for lt in xrange(lats):
            for ln in xrange(lons):
                yield (to_unix_timestamp(t), 
                       ds['lat'][lt], ds['lon'][ln], var[t,lt,ln])

def netcdf_dicts(var):
    for time, lat, lon, v in netcdf_items(var):
        yield {"time": time, "lat": lat,
               "lon": lon, "variable": v}

In [201]:
python_schema = {
    "namespace": "com.kitware.nex_gddp",
    "type": "record",
    "name": "GDDP Record",
    "fields": [
#        {"name": "key", "type": "string"},
        {"name": "variable", "type": "float"},
        {"name": "lat",  "type": "float"},
        {"name": "lon", "type": "float"},
        {"name": "time", "type": "long"},
     ]
}

schema = avro.schema.parse(json.dumps(python_schema))

writer = DataFileWriter(open("{}.avro".format(files[0].split(".")[0]), 
                             "w"), DatumWriter(), schema)

In [202]:
import sys

VAR='pr'
N_ROWS = reduce(lambda x, y: x*y, ds[VAR].shape)

t0 = time.time()
i = 0
for item in netcdf_dicts(ds[VAR]):
    writer.append(item)
    i = i + 1
    if (time.time() - t0) > 5:
        print "{}%".format((i / float(N_ROWS)) * 100)
        sys.out.flush()
        t0 = time.time()

writer.close()


AvroTypeException: The datum {'lat': -89.875, 'variable': 0.0, 'lon': 0.125, 'time': 851947200} is not an example of the schema {
  "namespace": "com.kitware.nex_gddp", 
  "type": "record", 
  "name": "GDDP Record", 
  "fields": [
    {
      "type": "float", 
      "name": "variable"
    }, 
    {
      "type": "float", 
      "name": "lat"
    }, 
    {
      "type": "float", 
      "name": "lon"
    }, 
    {
      "type": "long", 
      "name": "time"
    }
  ]
}